# 🚨 VERSIÓN 3: ENTRENAMIENTO CON VOCABULARIO PERSONALIZADO

In [ ]:
!pip install -q transformers[sentencepiece] datasets accelerate -U sentencepiece tokenizers sacremoses

import pandas as pd
from datasets import Dataset
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
import sentencepiece as spm
import os
import shutil
from google.colab import files

# 1️⃣ Carga de Datos

In [ ]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name, sep='|', quoting=3)

# 2️⃣ Entrenamiento de nuevos Tokenizers

In [ ]:
# Aquí puedes cambiar el tamaño del vocabulario (ej: 12000, 16000, 32000)
VOCAB_SIZE = 12000 

df["latin"].to_csv("latin.txt", index=False, header=False)
df["spanish"].to_csv("spanish.txt", index=False, header=False)

spm.SentencePieceTrainer.Train(input="latin.txt", model_prefix="source", vocab_size=VOCAB_SIZE, model_type="bpe")
spm.SentencePieceTrainer.Train(input="spanish.txt", model_prefix="target", vocab_size=VOCAB_SIZE, model_type="bpe")

# 3️⃣ Cargar Modelo Base y REDIMENSIONARlo

In [ ]:
MODEL_NAME = "Helsinki-NLP/opus-mt-itc-itc"
model = MarianMTModel.from_pretrained(MODEL_NAME)

# Creamos un tokenizer manual usando tus nuevos archivos .model
tokenizer = MarianTokenizer(
    source_spm="source.model",
    target_spm="target.model",
    source_lang="la",
    target_lang="es"
)

# 🔥 CRÍTICO: Ajustar el modelo al nuevo tamaño del tokenizer
model.resize_token_embeddings(len(tokenizer))
print(f"✅ Modelo redimensionado a {len(tokenizer)} tokens.")

# 4️⃣ Preparar Dataset BIDIRECCIONAL

In [ ]:
df_la_es = df.copy()
df_la_es["src"] = ">>es<< " + df_la_es["latin"].astype(str)
df_la_es["tgt"] = df_la_es["spanish"].astype(str)

df_es_la = df.copy()
df_es_la["src"] = ">>la<< " + df_es_la["spanish"].astype(str)
df_es_la["tgt"] = df_es_la["latin"].astype(str)

df_final = pd.concat([df_la_es[["src", "tgt"], df_es_la[["src", "tgt"]]], ignore_index=True)
print(f"✅ Dataset ampliado: {len(df_final)} filas.")

# 5️⃣ Preprocesamiento

In [ ]:
dataset = Dataset.from_pandas(df_final).shuffle(seed=42)
dataset_split = dataset.train_test_split(test_size=0.05)

def preprocess(examples):
    model_inputs = tokenizer(examples["src"], max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["tgt"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_data = dataset_split.map(preprocess, batched=True)

# 6️⃣ Entrenamiento

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./modelo_custom_vocab",
    per_device_train_batch_size=16,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer
)

print("🚀 Iniciando entrenamiento con vocabulario personalizado...")
trainer.train()

# 7️⃣ Guardado y ZIP

In [ ]:
model_dir = "./modelo_final_custom"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

shutil.make_archive("modelo_custom_completo", 'zip', model_dir)
files.download("modelo_custom_completo.zip")